In [101]:
import os 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import scipy.io 
import librosa 
import librosa.display 
import librosa as lr 
import wave
from glob import glob  #for reading files 
import glob 
import pickle as pk 

In [102]:
# path of the bird, session, .wav files 
path_bird = '/Users/ahmedabdalsattar/Desktop/bird12/'
path_session = os.listdir(path_bird)

# parameters given 
hop_length = 512
n_fft = 512
window_size = 1048
sampling_rate = 15000

In [103]:
# parameters given 
hop_length = 512
n_fft = 512
window_size = 1048
sampling_rate = 15000

data = {
    "session_num":[],
    "mfcc_vectors":[],
    "labels":[]

}

for session in path_session:
    if session == '.DS_Store': #sometimes this happen only for mac where DS_store automatically created 
        pass 
    else:
        wav_length = []
        short_wav_length = []
        mfcc_array = []
        session_array = os.path.join(path_bird,session)
        path_folder = session_array.strip("/").split("/")
        print("bird_name:{bird} session:{session}".format(bird=path_folder[-2],session=path_folder[-1]))
        data["session_num"].append(path_folder[-1])
        wave_files = glob.glob(os.path.join(session_array,'*.wav'))
        for i in range(len(wave_files)):
            audio_array,sr = lr.load(wave_files[i],sr=sampling_rate)
            time_period = lr.get_duration(filename=wave_files[i])
            if time_period >= 1:
                wav_length.append(time_period)
                audio_array,sr = lr.load(wave_files[i],sr=sampling_rate)
                mfcc = lr.feature.mfcc(audio_array, sr=sampling_rate,
                                            n_fft=n_fft,hop_length=hop_length)
                mfcc = mfcc.T
                mfcc_array.append(mfcc)
                data["mfcc_vectors"].append(mfcc_array)
            else:
                short_wav_length.append(time_period)
                pass 



bird_name:bird12 session:2020-08-13
bird_name:bird12 session:2020-08-12


In [104]:
"""for this function, i have to set a path for the folder where all labels are located
or find all the files that endwith .txt as it represent the labels
- for now the labels are all stored in one folder, include each .txt for each session"""

labels = []
path_labels = '/Users/ahmedabdalsattar/Desktop/labels/'

txt_files = os.listdir(path_labels)
labels = []
for file in range(len(txt_files)):
    path = os.path.join(path_labels,txt_files[file])
    a = os.path.basename(path[-1])
    if a == 't':
        x = open(path,"r")
        for line in x.readlines():
            for i in line.split():
                labels.append(int(i)) 
                
    

    else:
        pass
    
data["labels"].append(labels)

In [105]:
# with open('data_mfcc.txt','wb') as pickle_file:
#     pickle.dump(data,pickle_file)


# with open('data_mfcc.txt','rb') as pickle_file:
#     new_data = pickle.load(pickle_file)

In [143]:
# convert classes and labels into numpy arrays 
x = np.array(data["mfcc_vectors"])
y = np.array(data["labels"][0])

In [145]:
"""this function is to split the data into training and testing dataset"""

from sklearn.model_selection import train_test_split
inputs_train,inputs_test, targets_train,targets_test = train_test_split(x,y,test_size = 0.1)


In [149]:
import tensorflow.keras as keras 
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(x.shape)),
    
    keras.layers.Dense(512,activation="relu"),
    
    keras.layers.Dense(256,activation="relu"),
    
    keras.layers.Dense(64,activation="relu"),
    
    #output layer 
    keras.layers.Dense(10,activation="softmax")
     
])

In [150]:
optimizer = keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=optimizer,
             loss="sparse_categorical_crossentropy",
             metrics = ["accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 536)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               274944    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 423,370
Trainable params: 423,370
Non-trainable params: 0
_________________________________________________________________


In [155]:
model.fit(inputs_train,targets_train,
         validation_data=(inputs_test,targets_test),epochs=50,
          batch_size=32)